## Imports

In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, LSTM, Input, Concatenate

## Data

In [38]:
df = pd.read_csv('./data.csv')
data = df[['num1', 'num2']]
label = df['label'].values

data = data.values.reshape(400, 2)

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)
"""
# Split the data into training and testing sets
X_train = data[:, :320, :]  # Take first 320 samples for training
X_test = data[:, 320:, :]  # Take last 80 samples for testing
y_train = label[:320]  # Corresponding labels for training samples
y_test = label[320:]  # Corresponding labels for testing samples
"""
X_test.shape

(80, 2)

In [26]:
model = Sequential()
model.add(LSTM(32, input_shape=(400, 2)))
model.add(Dense(1))

In [39]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(data, label.values)
print("Test Accuracy:", accuracy)

Epoch 1/10


ValueError: in user code:

    File "c:\Users\huday\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\huday\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\huday\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\huday\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\huday\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   

In [61]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Read the CSV file into a DataFrame
df = pd.read_csv('./data.csv')

# Extract features and labels
data = df[['num1', 'num2']]
label = df['label'].values
                    #data size,time step ,number of feature each step
data = data.values.reshape(400, 2, 1)
#400,2,1

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

model = Sequential()
model.add(LSTM(32, input_shape=(2, 1)))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=20, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/25
13/13 [==============================] - 2s 40ms/step - loss: 0.7000 - accuracy: 0.6641 - val_loss: 0.6777 - val_accuracy: 0.6719
Epoch 2/25
13/13 [==============================] - 0s 5ms/step - loss: 0.6453 - accuracy: 0.6914 - val_loss: 0.6435 - val_accuracy: 0.6562
Epoch 3/25
13/13 [==============================] - 0s 5ms/step - loss: 0.6219 - accuracy: 0.6797 - val_loss: 0.6135 - val_accuracy: 0.6562
Epoch 4/25
13/13 [==============================] - 0s 6ms/step - loss: 0.5943 - accuracy: 0.6797 - val_loss: 0.5944 - val_accuracy: 0.6562
Epoch 5/25
13/13 [==============================] - 0s 5ms/step - loss: 0.5755 - accuracy: 0.6875 - val_loss: 0.5734 - val_accuracy: 0.6719
Epoch 6/25
13/13 [==============================] - 0s 5ms/step - loss: 0.5569 - accuracy: 0.6992 - val_loss: 0.5556 - val_accuracy: 0.6875
Epoch 7/25
13/13 [==============================] - 0s 6ms/step - loss: 0.5479 - accuracy: 0.6953 - val_loss: 0.5362 - val_accuracy: 0.6875
Epoch 8/25
13/13 [=

In [71]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Softmax

# Read the CSV file into a DataFrame
df = pd.read_csv('./data.csv')

# Extract features and labels
data = df[['num1', 'num2']]
label = df['label'].values
                    #data size,time step ,number of feature each step
data = data.values.reshape(400, 2, 1)
#400,2,1

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state=42)

model = Sequential()
model.add(LSTM(32, input_shape=(2, 1)))
model.add(Dense(1))
model.add(Dense(units=1, activation='ReLU'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=20, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

Epoch 1/100
13/13 [==============================] - 2s 37ms/step - loss: 4.2475 - accuracy: 0.3281 - val_loss: 1.2699 - val_accuracy: 0.3438
Epoch 2/100
13/13 [==============================] - 0s 5ms/step - loss: 1.1441 - accuracy: 0.3281 - val_loss: 0.9620 - val_accuracy: 0.3438
Epoch 3/100
13/13 [==============================] - 0s 5ms/step - loss: 0.9416 - accuracy: 0.3477 - val_loss: 0.8438 - val_accuracy: 0.4219
Epoch 4/100
13/13 [==============================] - 0s 5ms/step - loss: 0.8436 - accuracy: 0.3945 - val_loss: 0.7735 - val_accuracy: 0.4688
Epoch 5/100
13/13 [==============================] - 0s 5ms/step - loss: 0.7798 - accuracy: 0.4219 - val_loss: 0.7170 - val_accuracy: 0.4688
Epoch 6/100
13/13 [==============================] - 0s 5ms/step - loss: 0.7257 - accuracy: 0.4336 - val_loss: 0.6655 - val_accuracy: 0.4844
Epoch 7/100
13/13 [==============================] - 0s 5ms/step - loss: 0.6778 - accuracy: 0.4648 - val_loss: 0.6199 - val_accuracy: 0.6719
Epoch 8/100


In [2]:
df = pd.read_csv('./embeddings/combined_codes_updated.csv')
message_encoded = df['SmellKey']
message_encoded

0      0
1      0
2      0
3      0
4      0
      ..
595    1
596    1
597    1
598    1
599    1
Name: SmellKey, Length: 600, dtype: int64

In [3]:
# Combine the embeddings and encoded messages into a DataFrame
import torch
import numpy as np
embeddings = torch.load('embeddings/old_embeddings.pt')
embeddings2 = torch.load('embeddings/fixdd_embeddings.pt')
data = pd.DataFrame({
    'fixdd_embeddings.pt': embeddings.tolist(),
    'old_embeddings.pt': embeddings2.tolist(),
    'smellKey_encoded': message_encoded
})
data

,fixdd_embeddings.pt,old_embeddings.pt,smellKey_encoded
0,"[-0.2068517804145813, 0.4640653431415558, 0.37...","[-0.2068517804145813, 0.4640653431415558, 0.37...",0
1,"[-0.26910966634750366, 0.2898518145084381, 0.4...","[-0.26910966634750366, 0.2898518145084381, 0.4...",0
2,"[-0.3947080969810486, 0.31987395882606506, 0.4...","[-0.3947080969810486, 0.31987395882606506, 0.4...",0
3,"[-0.24552001059055328, 0.3665466904640198, 0.5...","[-0.24552001059055328, 0.3665466904640198, 0.5...",0
4,"[-0.26933154463768005, 0.4663805067539215, 0.4...","[-0.26933154463768005, 0.4663805067539215, 0.4...",0
...,...,...,...
595,"[-0.1658286303281784, -0.09777987748384476, 1....","[-0.1658286303281784, -0.09777987748384476, 1....",1
596,"[-0.1444222629070282, -0.1938023865222931, 1.1...","[-0.1444222629070282, -0.1938023865222931, 1.1...",1
597,"[0.08059106767177582, -0.23045925796031952, 2....","[0.08059106767177582, -0.23045925796031952, 2....",1
598,"[0.08911119401454926, -0.09515704959630966, 1....","[0.08911119401454926, -0.09515704959630966, 1....",1


In [49]:
x1 =np.vstack(data['fixdd_embeddings.pt'])
x2= np.vstack(data['old_embeddings.pt'])
X = np.concatenate((x1, x2), axis=1)

In [51]:

# Split the data into features (embeddings) and labels (messages) tensor

y = (data['smellKey_encoded'])
X= X.reshape(600, 2, 768)

tensor = tf.convert_to_tensor(X, dtype=tf.float32)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



AttributeError: EagerTensor object has no attribute 'reshape'. 
        If you are looking for numpy-related methods, please run the following:
        tf.experimental.numpy.experimental_enable_numpy_behavior()
      

In [7]:
model = Sequential()
model.add(LSTM(32, input_shape=(1, 2)))
model.add(Dense(1))
#model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=25, batch_size=20, validation_split=0.2)

# Evaluate the model

#X_train convert numpy to tensor




loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).